# Problem 2 - Swap and Connect
Whenever we run a quantum algorithm or circuit, we do not think of the underlying hardware. We are able to do this thanks to a set of steps that take place between the submission of our code and what the computer finally receives. These steps allow our **logical quantum circuit** to be transformed into a **physical quantum circuit** executable on a quantum backend. 

What do we mean by this? Let's take an example


<img src="resources/problem-2/backend-img.png">

- This is the picture of an actual 7-qubit backend **ibm_perth**. Each of the numbers represent a *qubit* and the edges between these nodes represent *connectivity*

- This is what is commonly represented as a **Graph**(https://en.wikipedia.org/wiki/Graph_theory) where the qubits are the vertices and edges are the connectivity between qubits.

- If you see qubits **0 and 1**, they are *directly connected*, but *not* the qubits **0 and 3**. This would pose some problems to our quantum circuit model of computation. What exactly? (try to think) ! 

If you are thinking of multi-qubit gates, you're right! 

- We learnt about the $CX$ gate and other 2 qubit gates during our lectures. While trying to implement these 2 qubit gates, it is often the case that the qubits **do not have a direct connectivity**

- Let us constrain ourselves for now and look at implementing the $CX$ gates. Let $CX(i,j)$ represent a $CX$ gate applied between qubits $i$ and $j$ with $i$ as the *control* qubit and $j$ as the *target* qubit 

- Continuing with our example above : 
    - $CX(1,3)$ is a directly possible operation
    - $CX(1,5)$ is not a directly possible operation 
    
- How do you connect two qubits which aren't directly connected? 

#### USE SWAP GATES!

- These figures below show a $CX(1,5)$ could be executed for the given graph above - 

<table><tr>
<td> <img src="resources/problem-2/no-swap-cx.png" /> </td>
<td> <img src="resources/problem-2/right-arrow.png" width = 100px height = 50px/></td>
<td> <img src="resources/problem-2/swap-cx.png"/> </td>
</tr></table>
    
- Since there is no direct connection between qubits 1 and 5, we can try to make a *bridge* through qubit 3. Similary if we wanted to make a connection between say qubits 0 and 6 we would need to make 3 such swap operations 

In [9]:
## Enter Team ID 
import os 
os.environ["TEAMID"] = "Excalibur"

## Task 
- Your task is to determine the minimum number of $SWAP$ gates needed to connect the qubits $i$ and $j$ in a given quantum backend. 
- You will be provided with the connectivity map of the device which is represented as a graph.
- This connectivity of the backend is provided in the form of a dictionary where each key represents a qubit  and their associated values are lists which contain nodes they are directly connected with. For example - 
```python 
#connectivity for above graph is like 
connectivity_map = {
    0 : [1], 
    1 : [0, 2, 3],
    2 : [1],
    3 : [1, 5],
    4 : [5],
    5 : [3, 4, 6],
    6 : [5]
}
```

## Level 1 - 30 points 

- You are given a connectivity map of a **line graph** . What this means is that you only have at most 2 neighbours for a particular vertex, and no cycles. For example - 

<img src = 'resources/problem-2/bogota.png'>

- The graph contains $N$ nodes and $N-1$ edges
- You are also given lists of control and target qubits with lengths $Q$

- You need to find the minimum number of swaps required to directly connect the control with its corresponding target qubit. For example - 


    
- **Constraints**
    - $1<=N<=100$
    - $1<=Q<=100$
    - Time limit for execution : 10 seconds


#### To submit 
- Create a function which should accept parameters `N`, `controls`, `targets` and `connectivity` 
- `N` is the total number of nodes in the graph
- `connectivity` represents the connectivity map of the given line graph
- `controls` and `targets` are going to be integer lists representing a list of controls and targets between which $CX$ gates are applied. 

- Your function should return an integer list `min_swaps` which represents the minimum number of swap operations required to connect the qubits `controls[i]` and `targets[i]` 

- We use $1-based$ indexing in the graph



**NOTE** 
1. Please refrain from adding any unneccessary kinds of statements other than comments in the designated code block, this may result in wrong output 
2. You can assume that the input connectivity map and the control and target qubits are all valid inputs

#### Example Test Case

```python
controls = [1, 2]
targets = [2, 5]
connectivity_map = {
    1 : [2],
    2 : [1,3],
    3 : [2,4],
    4 : [3,5],
    5 : [4]
}   
N = 5 

min_swaps = get_min_swaps_line(N, controls, targets, connectivity_map)

```
- `min_swaps = [0, 2]` as - 
    - $CX(1,2)$ requires 0 $SWAP$ gates
    - $CX(2,5)$ requires 2 $SWAP$ gates
    


In [ ]:
def get_min_swaps_line(N, controls, targets, connectivity_map):
    min_swaps = []
    ### You code goes here 
    length=len(controls)  
    for i in range(length):
        if(targets[i] in connectivity_map[controls[i]]):
            min_swaps.append(0)
        else:
            min_swaps.append(abs(targets[i]-controls[i])-1)
   
    
    ### your code goes here 
    
    return min_swaps

#### Test function
- Test your function before submitting to the grader 


In [11]:
def test_function_1():
    controls = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5]
    targets = [2,3,4,5,1,3,4,5,1,2,4,5,1,2,3,5,1,2,3,4]
    connectivity_map = {
        1 : [2],
        2 : [1,3],
        3 : [2,4],
        4 : [3,5],
        5 : [4]
    }   
    N = 5  

    min_swaps = get_min_swaps_line(N, controls, targets, connectivity_map)
    
    return min_swaps

test_function_1()

[0, 1, 2, 3, 0, 0, 1, 2, 1, 0, 0, 1, 2, 1, 0, 0, 3, 2, 1, 0]

In [12]:
from grader.graders.problem_2.grader import grader1
grader1.evaluate(get_min_swaps_line)

2022-10-17 13:16:18.492979
6 cells appended.
Congratulations, your answer is correct!


## Level 2 - 70 points

- You can see that the above connection is not very efficient. 
- Many qubits which are at the end points have to use swaps proportional to the size of the graph to connect with each other.

- We present graphs in a different connectivity now - 

<img src='resources/problem-2/big-graph.png'>

- Here the qubits are in a connected formm *but* each qubit may have more than 2 neighbours
- This introduces a larger connectivity range and minimizes the number of swaps to some extent. But a problem which arises now is how do we connect the qubits which are not directly connected? That is for you to solve!
- The graph contains $N$ nodes and $M$ edges where $M$ can be greater than $N-1$
- You are also given lists of control and target qubits with lengths $Q$
- You need to find the minimum number of swaps required to directly connect the control with its corresponding target qubit.

### Constraints
- The following constraints hold true :
    - $1 <= N <= 1000$
    - $1 <= M <= min(N(N-1)/2, 10000)$
    - $1 <= Q < 1000$
    - Time limit for execution : 60 seconds

#### To submit 
- Create a function which should accept parameters `N`, `M`, `controls`, `targets` and `connectivity` 
- `N` is the total number of nodes and `M` is the total number of edges in the graph
- `connectivity` represents the connectivity map of the given line graph
- `controls` and `targets` are going to be integer lists representing a list of controls and targets between which $CX$ gates are applied. 

- If the qubits cannot be connected, you should save `-1` as the answer for number of swaps required

- Your function should return an integer list `min_swaps` which represents the minimum number of swap operations required to connect the qubits `controls[i]` and `targets[i]` 
- We use $1-based$ indexing for the graph



**NOTE** 
1. Please refrain from adding any unneccessary kinds of statements other than comments in the designated code block, this may result in wrong output 
2. You can assume that the input connectivity map and the control and target qubits are all valid inputs

#### Example Test Case

```python
controls = [1, 2]
targets = [2, 5]
connectivity_map = {
    1 : [2],
    2 : [1,3],
    3 : [2,4,5],
    4 : [3],
    5 : [3]
}   
N = 5 
M = 4

min_swaps = get_min_swaps_graph(N, M, controls, targets, connectivity_map)

```
- `min_swaps = [0, 1]` as - 
    - $CX(1,2)$ requires 0 $SWAP$ gates
    - $CX(2,5)$ requires 1 $SWAP$ gate    


In [6]:
import sys
def get_min_swaps_graph(N, M, controls, targets, connectivity_map):
    min_swaps = []
    ### You code goes here
    length=len(controls)  
    for i in range(length):
        if(targets[i] in connectivity_map[controls[i]]):
            min_swaps.append(0)
        else:
            u=controls[i]
            v=targets[i]
            visited=[]
            distance=[]
            q=[]
            for i in range(N):
                visited.append(0)
                distance.append(sys.maxsize)
            for k in connectivity_map[u]:
                distance[k-1]=1
            distance[u-1]=0
            q.append(u)
            visited[u-1]=1
            while(q):
                x=q.pop(0)
                templist=connectivity_map[x]
                length2=len(templist)
                for j in range(length2):
                    if(visited[templist[j]-1]==1):
                        continue
                    if((distance[x-1]+1)<distance[templist[j]-1]):
                        distance[templist[j]-1]=distance[x-1]+1
                    q.append(templist[j])
                    visited[templist[j]-1]=1
            if(distance[v-1]==sys.maxsize):
                min_swaps.append(-1)
                continue
            min_swaps.append(distance[v-1]-1)        
    ### your code goes here 
    
    return min_swaps

### Test function
- Test your function before submitting to the grader 


In [14]:
def test_function_2():
    controls = [1, 2]
    targets = [2, 5]
    connectivity_map = {
        1 : [2],
        2 : [1,3],
        3 : [2,4,5],
        4 : [3],
        5 : [3]
    }   
    N = 5 
    M = 4
    min_swaps = get_min_swaps_graph(N, M, controls, targets, connectivity_map)
    
    return min_swaps

test_function_2()

[0, 1]

In [10]:
from grader.graders.problem_2.grader import grader2
grader2.evaluate(get_min_swaps_graph)

Failed on  /opt/.qbraid/environments/qbraid_000000/pyenv/lib/python3.9/site-packages/grader/graders/problem_2/./tests/inputs/task-2-0.txt
2022-10-19 17:07:37.783171
6 cells appended.
Uh-oh, that's not quite correct :(
